In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import csv
import pickle #to save notebook at sessions

from sklearn.preprocessing import RobustScaler

#set path for pickles to be saved in
pickle_path = '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/python pickles/'

In [4]:
print(pd.__version__)

2.0.1


# What is going on (on raw counts)
1. Import glycogene csv of human glycogenes converted to mice homologs (because our data is from mice tumor models. Human glycogenes are from glyco.me (Glycopacity). 
2. Load barcodes, features, and matrix of LN and TILs and convert to dataframe
3. Import transformed identity csv with info on which cells are associated with which cell type
4. Filter cells from original dataframe to produce smaller dataframe of just TILs in LN/TIL population from transformed identity csv
5. Filter genes from filtered dataframe (from 4) to only include glycogenes from (1)

## 1. Import human to mouse glycogene csv.
Converted human glycogenes to their mouse homologs via https://www.informatics.jax.org/batch/summary

In [9]:
# Load mouse glycogenes 

glyco_fp = 'human to mouse glycogene converted.csv'
glycogene_df = pd.read_csv(glyco_fp)

#make mouse glycogene list
m_glyco = glycogene_df[['Mouse']]
mouse_glyco_long = list(m_glyco.dropna(axis=0)['Mouse']) #removed 3 genes that didn't have mice homologs

#export just the mouse glycogenes as csv
m_glyco.to_csv('Mouse glycogenes.csv')

# remove duplicates
mouse_glycogenes = set(mouse_glyco_long)

In [6]:
#pickle list of mouse glycogenes
with open('mouse_glycogenes.pkl', 'wb') as f:
    pickle.dump(mouse_glycogenes, f)

## 2. Load barcodes, features, and matrix of LN and TILs and convert to dataframe


In [5]:
## TAKES LONG TIME TO RUN!! ## 
# Load the LN SUGAR-seq matrix from file
sparse_LN = sp.io.mmread(
    '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/LN data/matrix.mtx')
# Convert to CSR format
sparse_LN = sp.sparse.csr_matrix(sparse_LN)

In [6]:
## TAKES LONG TIME TO RUN!! ## 
# Load the TIL SUGAR-seq matrix from file
sparse_TIL = sp.io.mmread(
    '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/TIL data/matrix.mtx')
# Convert to CSR format
sparse_TIL = sp.sparse.csr_matrix(sparse_TIL)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/TIL data/matrix.mtx'

In [ ]:
# dimension check of sparse matrix
print('LN matrix dimesions are', np.shape(sparse_LN)) #31059 = rows , 21343=columns
print('TIL matrix dimesions are', np.shape(sparse_TIL))

# convert sparse matrices to array to put into dataframe
LN_matrix = sparse_LN.toarray()
TIL_matrix = sparse_TIL.toarray()

### Set up data frame for TIL

In [ ]:
### TIL data ##  
# barcodes = row names
barcodes = pd.read_csv(
    '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/TIL data/barcodes copy.tsv', 
    header=None)
barcodes = barcodes.rename(columns={0: 'barcode'})
TIL_barcodes = list(barcodes['barcode']) # of columns =  21343

# features = column names
feats = pd.read_csv(
    '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/TIL data/features.tsv', 
    header = None)
with open('/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/TIL data/features.tsv', 
          'r') as infile:
    reader = csv.reader(infile, delimiter='\t')
    feat_ls = []
    for row in reader:
        feat_ls.append(row)
TIL_features = [i[1] for i in feat_ls] # of rows = 31059

# Convert the dense matrix to a pandas DataFrame
TIL_df = pd.DataFrame(data=TIL_matrix, index=TIL_features, columns=TIL_barcodes)

# Remove rows for HTO1, HTO2, PD1, and TIM3 (ADT and HTO assays)
TIL_df_new = TIL_df.drop(['HTO1', 'HTO2','PD1', 'TIM3'])

### set up dataframe for LN

In [ ]:
## LN data ## 
# barcodes = row names
barcodes = pd.read_csv(
    '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/LN data/barcodes copy.tsv', 
    header=None)
barcodes = barcodes.rename(columns={0: 'barcode'})
LN_barcodes = list(barcodes['barcode']) # of columns =  21343

# features = column names
feats = pd.read_csv(
    '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/LN data/features.tsv', header = None)
with open('/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/raw data/LN data/features.tsv', 
          'r') as infile:
    reader = csv.reader(infile, delimiter='\t')
    feat_ls = []
    for row in reader:
        feat_ls.append(row)
LN_features = [i[1] for i in feat_ls] # of rows = 31059

# Convert the dense matrix to a pandas DataFrame with columns as each barcode and rows as each gene
LN_df = pd.DataFrame(data=LN_matrix, index=LN_features, columns=LN_barcodes)

# Remove rows for HTO1, HTO2, PD1, and TIM3 (ADT and HTO assays)
LN_df_new = LN_df.drop(['hashtag1', 'hashtag2','hashtag3', 'PD1_hash', 'TIM3_hash'])

In [ ]:
with open(pickle_path + 'TIL df set up.pkl', 'wb') as f:
    pickle.dump(TIL_df_new, f)
    
with open(pickle_path + 'LN df set up.pkl', 'wb') as f:
    pickle.dump(LN_df_new, f)

f.close()

## 3. Add cell type info from csv exported from ProjecTIL cell type annotation

In [5]:
with open(pickle_path + 'TIL df set up.pkl', 'rb') as f:
    TIL_df_new = pickle.load(f)

with open(pickle_path + 'LN df set up.pkl', 'rb') as f:
    LN_df_new = pickle.load(f)

In [6]:
#check to see St6galnac2 -->  non-identical columns but named the same 
n = TIL_df_new.transpose()
n = n.filter(regex='^St6galnac2$')
names = ['1','2']
n.columns = names
n
print(n['1'].equals(n['2']))

False


In [7]:
path_TIL='/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/TIL output/ProjecTIL analysis/TIL_transformed_identity.csv'
path_LN='/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/LN output/ProjecTIL analysis/LN_transformed_identity.csv'

# import ProjecTIL analysis output to add cell type annotation to raw glycogene data frame 
TIL_transformed_ident = pd.read_csv(path_TIL)
LN_transformed_ident = pd.read_csv(path_LN)

#rename column of ProjecTIL df for easy identification of barcodes
TIL_transformed_ident = TIL_transformed_ident.rename(columns={'Unnamed: 0':'Barcodes'}) 
LN_transformed_ident = LN_transformed_ident.rename(columns={'Unnamed: 0':'Barcodes'}) 

#Transpose matrix to match final TIL/LN matrix format
TIL_transformed_ident = TIL_transformed_ident.transpose()
LN_transformed_ident = LN_transformed_ident.transpose()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/erikazhang/Dropbox (MIT)/20.440 Biological Networks/project/TIL output/ProjecTIL analysis/TIL_transformed_identity.csv'

In [10]:
#RUn only once, else it keeps replacing index with top row. if you run too much, rerun previous cell first!! 
'''
make the column names the barcodes, which is how the glycogene 
dataframe (the one we want to combine this one with) is formatted as, to concatenate
'''
TIL_transformed_ident.set_axis(TIL_transformed_ident.iloc[0], axis=1, inplace=True)
TIL_transformed_ident = TIL_transformed_ident[1:] 

LN_transformed_ident.set_axis(LN_transformed_ident.iloc[0], axis=1, inplace=True)
LN_transformed_ident = LN_transformed_ident[1:]

In [ ]:
#save as binarized pickle file 
with open(pickle_path + 'TIL_transformed_ident.pkl', 'wb') as f:
    pickle.dump(TIL_transformed_ident, f)
    
with open(pickle_path + 'LN_transformed_ident.pkl', 'wb') as f:
    pickle.dump(LN_transformed_ident, f)

f.close()

### Remove non-TILs from dataframe

In [12]:
# # filter original matrix by barcodes of cells identified as TILs via ProjecTILs #

# extract list of barcodes that have a T-cell annotation
TIL_barcodes = list(TIL_transformed_ident.columns)
LN_barcodes = list(LN_transformed_ident.columns)


filtered_TIL_df = TIL_df_new[TIL_barcodes]
filtered_LN_df = LN_df_new[LN_barcodes]

print('Original dataframe dimensions are ', TIL_df_new.shape, 
      'and the filtered dataframe dimensions are', filtered_TIL_df.shape)
print('Original dataframe dimensions are ', LN_df_new.shape, 
      'and the filtered dataframe dimensions are', filtered_LN_df.shape)

Original dataframe dimensions are  (31054, 19912) and the filtered dataframe dimensions are (31054, 19645)
Original dataframe dimensions are  (31054, 21343) and the filtered dataframe dimensions are (31054, 21296)


In [ ]:
#save as binarized pickle file 
with open(pickle_path + 'filtered_TIL_df.pkl', 'wb') as f:
    pickle.dump(filtered_TIL_df, f)
    
with open(pickle_path + 'filtered_LN_df.pkl', 'wb') as f:
    pickle.dump(filtered_LN_df, f)

f.close()

In [ ]:
##TAKES LONG TIME TO RUN##
'''
Add cell type information via concatenation
Note: the L-Pha information from this csv is normalized, and since we're using raw data we'll use the raw biotin info
'''
TIL_type = TIL_transformed_ident.loc[['Type']]
LN_type = LN_transformed_ident.loc[['Type']]

#makes new dataframe that adds T-cell type annotations to raw gene expression dataframe 
TIL_type_df = pd.concat([filtered_TIL_df, TIL_type])
LN_type_df = pd.concat([filtered_LN_df, LN_type])

In [ ]:
with open(pickle_path + 'TIL_type_df.pkl', 'wb') as f:
    pickle.dump(TIL_type_df, f)
    
with open(pickle_path + 'LN_type_df.pkl', 'wb') as f:
    pickle.dump(LN_type_df, f)

f.close()

## 4. DO normalization of T-cells via full transcriptome 

In [4]:
with open('TIL_type_df.pkl', 'rb') as f:
    TIL_type_df = pickle.load(f)

# with open('LN_type_df.pkl', 'rb') as f:
#     LN_type_df = pickle.load(f)

In [4]:
LN_type_df

,AAACCTGAGAATTGTG-1,AAACCTGAGAGACTAT-1,AAACCTGAGAGGGATA-1,AAACCTGAGATCTGAA-1,AAACCTGAGCCTATGT-1,AAACCTGAGCGTAGTG-1,AAACCTGAGCTCCTTC-1,AAACCTGAGGCGATAC-1,AAACCTGAGGGATCTG-1,AAACCTGAGGGTCGAT-1,...,TTTGTCAGTGTTCGAT-1,TTTGTCAGTTTGTTTC-1,TTTGTCATCAACACCA-1,TTTGTCATCAATAAGG-1,TTTGTCATCAGAGCTT-1,TTTGTCATCCGCATCT-1,TTTGTCATCGGCATCG-1,TTTGTCATCTAACCGA-1,TTTGTCATCTTAACCT-1,TTTGTCATCTTGAGAC-1
Xkr4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Gm1992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Gm37381,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Rp1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sox17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CAAA01118383.1,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,1,0,1,0,0,0
Vmn2r122,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CAAA01147332.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Biotin_hash,223,124,404,138,1080,232,252,173,186,536,...,236,86,149,95,140,159,196,152,88,124


In [7]:
'''
Perform robust scaler normalization

This defines a function that normalizes expression across the different genes for each cell.

input:
- df with last 2 rows that are type, biotin
- rows = genes, columns = cells

output:
- scaled df with same dimensions
- df with columns as genes and rows as cells
'''

def robust_normalization(df):
    ## isolate just gene columns and standardize 
    toy_df = df.iloc[:-2]
    cols_to_standardize = toy_df.columns
    scaler = RobustScaler()
    toy_df[cols_to_standardize] = scaler.fit_transform(toy_df[cols_to_standardize])
    
    ## extract biotin, t-cell type, and L-PHA data from original dataframe and append to transformed data
    scores_to_append = df.iloc[-2:]

    norm_df = pd.concat([toy_df, scores_to_append])
    norm_df = norm_df.transpose()
   
    return norm_df  

In [21]:
'''
Now want to add Glycosylation (PHA-L) column that assigns:
Yes (1) to top 25%, 
No (0) to bottom 25%

categorize_lectin 
inputs:
- data_all: a dataframe that has
    - genes in the columns headers
    - barcodes as row indices
    - a column with biotin values to use as cutoffs for top 25% and bottom 25%
- quantile_high: upper quantile threshold (75% to represent top 25% of data below which 75% of the data falls)
- quantile_low: lower quantile threshold (25% to represent bottom 25% of data below which 25% of data falls)
- ref_col_loc: index of column in data_all that is used for categorization cutoffs

outputs:
- cutoff values for high and low biotin
- list of 3 arrays representing indices of rows that meet the high biotin cutoff, 
low biotin cutoff, and combined high and low cutoff
- number of rows that meet the high category cutoff and the low category cutoff.
'''

# Function: determine PHA-L read cut-offs for binary classification FROM BOJAR LAB
def categorize_lectin(data_all, quantile_high, quantile_low, ref_col_loc):
    cutoff = np.quantile(data_all.loc[:,ref_col_loc], [quantile_high, quantile_low], interpolation="nearest").tolist()

    print(f"Cut-off for PHA-L high: {cutoff[0]}; Cut-off for PHA-L low: {cutoff[1]}")

    high_indices = np.array(data_all.loc[:,ref_col_loc]>=cutoff[0])
    low_indices = np.array(data_all.loc[:,ref_col_loc]<cutoff[1])
    high_low_indices = np.logical_or(high_indices, low_indices)

    high_count = high_indices.sum()
    low_count = low_indices.sum()

    return cutoff, [high_indices, low_indices, high_low_indices], [high_count, low_count]


'''
Function assigns binary glycoscore to input df

input:
- c = dataframe containing normalized counts of gene expression per cell
- Need last 2 columns to be 'Type' and 'Biotin' col
- genes on column, cell barcodes on row 

output:
- dataframe containing new column for L-PHA score 
- genes on column, barcodes on row 
'''

def glycoscore(c):
    #sort dataframe by 'Biotin values'
    c = c.sort_values(by='Biotin', ascending=False)
    # Parameters for categorize lectin function
    quantile_high, quantile_low = 0.75, 0.25
    ref_col = 'Biotin' #last column of dataframes contain biotin info

    #split df into quartiles 
    cutoff, indices, count = categorize_lectin(c, quantile_high, quantile_low, ref_col)

    # Assign 1 to top 25% and 0 to bottom 25%
    c.loc[indices[0], "PHA-L"] = 1
    c.loc[indices[1], "PHA-L"] = 0

    # Drop the middle two quartiles
    c = c.loc[indices[2], :]
    return c

In [ ]:
n = TIL.transpose(a)
n = n.filter(regex='^St6galnac2$')
n

In [9]:
'''
Normalize gene counts across all genes for each T-cell

recall robust normalization will spit out TRANSPOSED dataframe
TAKES FOREVER
'''
normTIL = robust_normalization(TIL_type_df)
# normLN = robust_normalization(LN_type_df)

/tmp/ipykernel_2727/106849003.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toy_df[cols_to_standardize] = scaler.fit_transform(toy_df[cols_to_standardize])


In [9]:
# Rename Biotin_hash column in LN dataframe to 'Biotin' for easy glycoscoring via glycoscore function
pre_scoredLN = normLN.rename(columns={'Biotin_hash': 'Biotin'})

In [10]:
with open('normTIL_full.pkl', 'wb') as f:
    pickle.dump(normTIL, f)

# with open('normLN_full.pkl', 'wb') as f:
#     pickle.dump(pre_scoredLN, f)

# pickle_in = open("normTIL_full.pkl","rb")
# normTIL_full = pickle.load(pickle_in)
# pickle_in = open("normLN_full.pkl","rb")
# normLN_full = pickle.load(pickle_in)

## Glycoscoring of normalized dfs

In [9]:
# Assign glycoscores to each cell
#TIL_glyconorm = = glycoscore(pre_scoredTIL)
LN_glyconorm = glycoscore(pre_scoredLN)

/tmp/ipykernel_7394/515535515.py:25: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  cutoff = np.quantile(data_all.loc[:,ref_col_loc], [quantile_high, quantile_low], interpolation="nearest").tolist()


Cut-off for PHA-L high: 276; Cut-off for PHA-L low: 127


In [10]:
# save pickle of df
# with open('full_TIL_glyconorm.pkl', 'wb') as f:
#     pickle.dump(TIL_glyconorm, f)

with open('full_LN_glyconorm.pkl', 'wb') as f:
    pickle.dump(LN_glyconorm, f)

## 5. Filter matrix to only include glycogenes (incl. housekeeping)

In [6]:
'''
These dataframes have genes+biotin+type+glycoscore in columns and cells in rows
'''
#from JEssie
with open('full_TIL_glyconorm.pkl', 'rb') as f:
    TIL_type_df = pickle.load(f)

with open('full_LN_glyconorm.pkl', 'rb') as f:
    LN_type_df = pickle.load(f)

In [7]:
TIL = TIL_type_df.transpose()
LN = LN_type_df.transpose()

In [21]:
'''
filter the original matrix by the mouse glycogenes but keep biotin and type info

NOTE: input dataframe must have genes in rows and cells in columns
'''
print('Total number of glycogenes looked for:', len(mouse_glycogenes))

#TIL
TILglycogenes_found = [i for i in mouse_glycogenes if i in TIL.index]
TILglycogenes_notfound = [i for i in mouse_glycogenes if i not in TIL.index]

#filter by glycogenes while keeping type&biotin&glycoscore
glycoTIL = TIL.loc[TILglycogenes_found + ['Biotin', 'Type', 'PHA-L']] 
print('Number of glycogenes found in TILs:', len(glycoTIL_df)-3) #-3 accounts for extra type, biotin, PHA-L rows


#LN
LNglycogenes_found = [i for i in mouse_glycogenes if i in LN.index]
LNglycogenes_notfound = [i for i in mouse_glycogenes if i not in LN.index]

glycoLN = LN.loc[LNglycogenes_found + ['Biotin', 'Type', 'PHA-L']] 
print('Number of glycogenes found in LNs:', len(glycoLN_df) - 3)

Total number of glycogenes looked for: 245
Number of glycogenes found in TILs: 9821
Number of glycogenes found in LNs: 241


In [22]:
glycoTIL_df = glycoTIL.transpose()
glycoLN_df = glycoLN.transpose()
glycoTIL_df

,Galnt13,B3gnt2,Gcnt2,A4galt,St3gal4,St8sia1,Extl2,Plod3,B3gnt7,Dpy19l1,...,Chpf2,Mgat4d,Bcap31,Pomt2,St8sia2,Alg10b,Piga,Biotin,Type,PHA-L
TACAGTGGTTCAACCA-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,37384,CD8_NaiveLike,1.0
ACATACGAGTGCCATT-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33249,CD8_EffectorMemory,1.0
TGTCCCATCGGCTTGG-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,32029,Th1,1.0
ACTGTCCAGGATGGTC-1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,31700,CD8_EarlyActiv,1.0
GAACATCTCTGCTGTC-1,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,31402,Treg,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CGTCAGGGTTTGACAC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15,CD8_NaiveLike,0.0
ACGGGTCTCGAACTGT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13,CD8_EffectorMemory,0.0
GCTGCTTCACCAACCG-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,CD8_NaiveLike,0.0
TGGTTCCGTTGATTCG-1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,CD8_NaiveLike,0.0


In [23]:
'''
These dataframes are glycoscored and normalized on full, 
then filtered to only include glycogenes

they have genes+biotin+type+score in columns and cells in rows
'''

with open('glycoTIL_df.pkl', 'wb') as f:
    pickle.dump(glycoTIL_df, f)
    
with open('glycoLN_df.pkl', 'wb') as f:
    pickle.dump(glycoLN_df, f)

f.close()

In [ ]:
'''
Couldn't find all the genes from glycogene list in dataset, see if we're missing any homologs etc...
THe same glycogenes that weren't found in TILs were also not found in LN
'''
TILnot = [i for i in mouse_glycogenes not in glycoTIL_df.index]
TILnot

# 7. Split normalized df  into T-cell subsets

SPLIT data from normLN_full and normLN_full dataframes (pre-glyco score, post full normalization, not glycofiltered) 
by t-cell subtype
store the dataframes into dictionary with t-cell type name as the key 

normTIL_full and normLN_full have genes + type + biotin in rows and cells in columns

Need normalized, glycogene filtered, but not scored dataframes 
glycoscores affected by number of cells



In [2]:
'''
GLycogene filtering on non-scored, normalized df
normTIL_full and normLN_full have genes, biotin, type in columns and genes in rows
'''
# pickle_in = open("normTIL_full.pkl","rb")
# normTIL_full = pickle.load(pickle_in)
pickle_in = open("normTIL_full.pkl","rb")
normTIL_full = pickle.load(pickle_in)

pickle_in = open("normLN_full.pkl","rb")
normLN_full = pickle.load(pickle_in)

In [12]:
TIL = normTIL_full.transpose()
LN = normLN_full.transpose()

In [13]:
'''
filter the original matrix by the mouse glycogenes but keep biotin and type info

NOTE: input dataframe must have genes in rows and cells in columns
'''
print('Total number of glycogenes looked for:', len(mouse_glycogenes))


#TIL
TILglycogenes_found = [i for i in mouse_glycogenes if i in TIL.index]
TILglycogenes_notfound = [i for i in mouse_glycogenes if i not in TIL.index]

#filter by glycogenes while keeping type&biotin
glycoTIL_df = TIL.loc[TILglycogenes_found + ['Biotin', 'Type']] 
print('Number of glycogenes found in TILs:', len(glycoTIL_df)-2) #-2 accounts for extra type and biotin rows


#LN
LNglycogenes_found = [i for i in mouse_glycogenes if i in LN.index]
LNglycogenes_notfound = [i for i in mouse_glycogenes if i not in LN.index]
#filter by glycogenes while keeping type&biotin
glycoLN_df = LN.loc[LNglycogenes_found + ['Biotin', 'Type']]
print('Number of glycogenes found in LNs:', len(glycoLN_df) - 2)

Total number of glycogenes looked for: 245
Number of glycogenes found in TILs: 241
Number of glycogenes found in LNs: 241


In [26]:
'''
TIL and LN needs to have genes in columns and cells in row to split by T-cell subtype
'''
#transpose back to get cells in row and genes in column
TIL = glycoTIL_df.transpose()
LN = glycoLN_df.transpose()

# Get list of T-cell subtypes for which to make a dataframe for TILs
tcell_subtypes = TIL['Type'].unique()

#make dataframe names for later access
df_names = [i+'_df' for i in list(tcell_subtypes)]

# Make copy of original df just in case
splitTIL = TIL.copy()
splitLN = LN.copy()


TILnorm_dfs_sub = {}
LNnorm_dfs_sub = {}

# Make separate dataframe containing data for each t-cell subtype
for cell_type, name in zip(tcell_subtypes, df_names):
    TILnorm_dfs_sub[name] = splitTIL[splitTIL['Type'] == cell_type]
    LNnorm_dfs_sub[name] = splitLN[splitLN['Type'] == cell_type]

In [4]:
# # save pickle of dictionary containing df for each subtype that is normalized but NOT glycoscored 
# with open('TILnorm_dfs_sub.pkl', 'wb') as f:
#     pickle.dump(TILnorm_dfs_sub, f)

# with open('LNnorm_dfs_sub.pkl', 'wb') as f:
#     pickle.dump(LNnorm_dfs_sub, f)

# # f.close()

'''load dictionary containing dataframes of non-normalized glycogene dataframes of just T-cells'''
pickle_in = open("TILnorm_dfs_sub.pkl","rb")
TILtcell_dfs_sub = pickle.load(pickle_in)
pickle_in = open( "LNnorm_dfs_sub.pkl","rb")
LNtcell_dfs_sub = pickle.load(pickle_in)

## 7B. Add glycoscores to each subtype df

In [22]:
TILglyconorm_sub_df = {}
LNglyconorm_sub_df = {}

# Make separate dataframe containing data for each t-cell subtype
for name in df_names:
    TILscored = glycoscore(TILtcell_dfs_sub[name])
    TILglyconorm_sub_df[name] = TILscored
    LNscored = glycoscore(LNtcell_dfs_sub[name])
    LNglyconorm_sub_df[name] = LNscored

/tmp/ipykernel_2923/515535515.py:25: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  cutoff = np.quantile(data_all.loc[:,ref_col_loc], [quantile_high, quantile_low], interpolation="nearest").tolist()
/tmp/ipykernel_2923/515535515.py:25: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  cutoff = np.quantile(data_all.loc[:,ref_col_loc], [quantile_high, quantile_low], interpolation="nearest").tolist()
/tmp/ipykernel_2923/515535515.py:25: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lo

Cut-off for PHA-L high: 1426; Cut-off for PHA-L low: 519
Cut-off for PHA-L high: 2011; Cut-off for PHA-L low: 579
Cut-off for PHA-L high: 416; Cut-off for PHA-L low: 187
Cut-off for PHA-L high: 330; Cut-off for PHA-L low: 121
Cut-off for PHA-L high: 506; Cut-off for PHA-L low: 233
Cut-off for PHA-L high: 349; Cut-off for PHA-L low: 150
Cut-off for PHA-L high: 403; Cut-off for PHA-L low: 205


/tmp/ipykernel_2923/515535515.py:25: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  cutoff = np.quantile(data_all.loc[:,ref_col_loc], [quantile_high, quantile_low], interpolation="nearest").tolist()
/tmp/ipykernel_2923/515535515.py:25: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  cutoff = np.quantile(data_all.loc[:,ref_col_loc], [quantile_high, quantile_low], interpolation="nearest").tolist()
/tmp/ipykernel_2923/515535515.py:25: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lo

Cut-off for PHA-L high: 253; Cut-off for PHA-L low: 126
Cut-off for PHA-L high: 1240; Cut-off for PHA-L low: 465
Cut-off for PHA-L high: 1691; Cut-off for PHA-L low: 624
Cut-off for PHA-L high: 648; Cut-off for PHA-L low: 231
Cut-off for PHA-L high: 400; Cut-off for PHA-L low: 133
Cut-off for PHA-L high: 873; Cut-off for PHA-L low: 297
Cut-off for PHA-L high: 937; Cut-off for PHA-L low: 343
Cut-off for PHA-L high: 325; Cut-off for PHA-L low: 162
Cut-off for PHA-L high: 182; Cut-off for PHA-L low: 105
Cut-off for PHA-L high: 500; Cut-off for PHA-L low: 215
Cut-off for PHA-L high: 379; Cut-off for PHA-L low: 182


/tmp/ipykernel_2923/515535515.py:25: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  cutoff = np.quantile(data_all.loc[:,ref_col_loc], [quantile_high, quantile_low], interpolation="nearest").tolist()
/tmp/ipykernel_2923/515535515.py:25: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
  cutoff = np.quantile(data_all.loc[:,ref_col_loc], [quantile_high, quantile_low], interpolation="nearest").tolist()
/tmp/ipykernel_2923/515535515.py:25: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lo

In [36]:
'''
save pickle of dictionary containing dataframe for each t-cell subtype normalized and glycoscored
- with genes in cols, cells in rows
'''
with open('TILglyconorm_sub_df.pkl', 'wb') as f:
    pickle.dump(TILglyconorm_sub_df, f)

with open('LNglyconorm_sub_df.pkl', 'wb') as f:
    pickle.dump(LNglyconorm_sub_df, f)

f.close()

# '''load dictionary containing dataframes of non-normalized glycogene dataframes of just T-cells'''
# pickle_in = open(pickle_path + "TILglyconorm_sub_df.pkl","rb")
# TILglyconorm_sub_df = pickle.load(pickle_in)
# pickle_in = open(pickle_path + "LNglyconorm_sub_df.pkl","rb")
# LNglyconorm_sub_df = pickle.load(pickle_in)

In [10]:
pickle_in = open("TILglyconorm_sub_df.pkl","rb")
TILglyconorm_sub_df = pickle.load(pickle_in)
pickle_in = open("LNglyconorm_sub_df.pkl","rb")
LNglyconorm_sub_df = pickle.load(pickle_in)

In [22]:
subtype_comp= pd.read_csv('subtype & cross model testing.csv')
sub = subtype_comp[['Data', 'Dimensions']]